In [14]:
# roc_analysis.py
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from pycaret.classification import load_model

def analyze_roc(model_path, test_data_path, output_dir="../Docs/Imagens"):
    """
    Gera análise da curva ROC e determina o threshold ideal.
    
    Args:
        model_path: Caminho para o modelo treinado (.pkl)
        test_data_path: Caminho para os dados de teste (.parquet)
        output_dir: Pasta para salvar o gráfico
    """
    # Carregar modelo e dados
    model = load_model(model_path)
    df_test = pd.read_parquet(test_data_path)
    
    # Extrair features e target
    X_test = df_test.drop(columns="shot_made_flag")
    y_true = df_test["shot_made_flag"]
    
    # Obter probabilidades preditas
    y_proba = model.predict_proba(X_test)[:, 1]
    
    # Calcular métricas ROC
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)
    
    # Encontrar threshold ideal pelo índice de Youden
    youden_idx = (tpr - fpr).argmax()
    optimal_threshold = thresholds[youden_idx]
    
    # Plotar curva ROC
    plt.figure(figsize=(10, 6))
    RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc).plot()
    plt.plot(fpr[youden_idx], tpr[youden_idx], 'ro', 
             label=f'Threshold Ideal ({optimal_threshold:.2f})')
    plt.title(f'Curva ROC (AUC = {roc_auc:.2f})')
    plt.legend(loc='lower right')
    
    # Salvar gráfico
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(f"{output_dir}/curva_roc.png")
    plt.close()
    
    return optimal_threshold, roc_auc

if __name__ == "__main__":
    threshold, auc_score = analyze_roc(
        model_path="../Data/Modeling/modelo_final",
        test_data_path="../Data/Processed/base_test.parquet"
    )
    
    print(f"Threshold Ideal: {threshold:.2f}")
    print(f"AUC Score: {auc_score:.2f}")
    print("Gráfico salvo em: Docs/Imagens/curva_roc.png")

Transformation Pipeline and Model Successfully Loaded
Threshold Ideal: 0.56
AUC Score: 0.59
Gráfico salvo em: Docs/Imagens/curva_roc.png


<Figure size 1000x600 with 0 Axes>